# Lyricist ai

## Load data

In [3]:
import re    
import glob
import numpy as np         
import tensorflow as tf    
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


## Data cleaning

In [5]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx >30: break   # 일단 문장 20개만 확인해 볼 겁니다.
    
    print(sentence)

Busted flat in Baton Rouge, waitin' for a train
And I's feelin' near as faded as my jeans
Bobby thumbed a diesel down, just before it rained
It rode us all the way to New Orleans I pulled my harpoon out of my dirty red bandanna
I was playin' soft while Bobby sang the blues, yeah
Windshield wipers slappin' time, I was holdin' Bobby's hand in mine
We sang every song that driver knew Freedom's just another word for nothin' left to lose
Nothin', don't mean nothin' hon' if it ain't free, no no
And, feelin' good was easy, Lord, when he sang the blues
You know, feelin' good was good enough for me
Good enough for me and my Bobby McGhee From the Kentucky coal mine to the California sun
There Bobby shared the secrets of my soul
Through all kinds of weather, through everything we done
Yeah, Bobby baby kept me from the cold One day up near Salinas, Lord, I let him slip away
He's lookin' for that home, and I hope he finds it
But, I'd trade all of my tomorrows, for a single yesterday
To be holdin' B

## Delete 특수문자

In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()   
    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
     # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r'[" "]+', " ", sentence) 
     # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 

    sentence = sentence.strip()
    # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    sentence = '<start> ' + sentence + ' <end>'      
    
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))   

<start> this is sample sentence . <end>


In [7]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> it rode us all the way to new orleans i pulled my harpoon out of my dirty red bandanna <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> we sang every song that driver knew freedom s just another word for nothin left to lose <end>',
 '<start> nothin , don t mean nothin hon if it ain t free , no no <end>',
 '<start> and , feelin good was easy , lord , when he sang the blues <end>',
 '<start> you know , feelin good was good enough for me <end>']

## 데이터 토큰화

In [8]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
	
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, 
                                                           padding='post',
                                                          maxlen=maxlen)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 3604 1691 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2  804    1 ...    0    0    0]
 ...
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f36aaa97e80>


In [9]:
# 단어 사전 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [10]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2 3604 1691   14    1 3995    4 1100   28    9  681    3    0    0
    0    0    0    0    0]
[3604 1691   14    1 3995    4 1100   28    9  681    3    0    0    0
    0    0    0    0    0]


## Make train, test set

In [18]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (140599, 19)
Target Train:  (140599, 19)


In [17]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

## Make AI model

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [20]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 7001), dtype=float32, numpy=
array([[[-2.35608546e-04, -7.64364231e-05,  1.16892450e-04, ...,
          1.90172708e-04, -3.65676620e-04,  9.89767068e-05],
        [-1.19571458e-04, -3.14243189e-05,  4.00590943e-04, ...,
          4.10022010e-04, -7.87389290e-04,  1.90423583e-04],
        [ 8.81598316e-05, -2.61214271e-04,  5.67055074e-04, ...,
          2.77251762e-04, -1.19517371e-03,  1.79567833e-05],
        ...,
        [ 1.67099421e-03, -2.55617383e-03, -1.04821078e-03, ...,
         -1.90157164e-03,  2.37548654e-03,  1.07440643e-03],
        [ 1.84452103e-03, -2.69731833e-03, -1.10692589e-03, ...,
         -1.84870732e-03,  2.46710144e-03,  1.28259964e-03],
        [ 1.99557515e-03, -2.82869814e-03, -1.16013386e-03, ...,
         -1.79553591e-03,  2.53922655e-03,  1.46862795e-03]],

       [[-2.35608546e-04, -7.64364231e-05,  1.16892450e-04, ...,
          1.90172708e-04, -3.65676620e-04,  9.89767068e-05],
        [-2.48197641e-04,  1.38534320e-04,  1.

In [21]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  1792256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


## Evaluate model performance

In [22]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
686/686 [==============================] - 125s 179ms/step - loss: 2.7904
Epoch 2/30
686/686 [==============================] - 128s 186ms/step - loss: 2.3833
Epoch 3/30
686/686 [==============================] - 128s 186ms/step - loss: 2.2380
Epoch 4/30
686/686 [==============================] - 127s 185ms/step - loss: 2.1341
Epoch 5/30
686/686 [==============================] - 128s 186ms/step - loss: 2.0475
Epoch 6/30
686/686 [==============================] - 128s 186ms/step - loss: 1.9690
Epoch 7/30
686/686 [==============================] - 128s 187ms/step - loss: 1.8952
Epoch 8/30
686/686 [==============================] - 128s 187ms/step - loss: 1.8244
Epoch 9/30
686/686 [==============================] - 128s 186ms/step - loss: 1.7570
Epoch 10/30
686/686 [==============================] - 128s 187ms/step - loss: 1.6912
Epoch 11/30
686/686 [==============================] - 129s 187ms/step - loss: 1.6271
Epoch 12/30
686/686 [==============================] - 128s 186

In [23]:
results = model.evaluate(enc_val,  dec_val, verbose=2)

print(results)

1099/1099 - 14s - loss: 0.7723
0.7722645401954651


In [24]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love the way you lie <end> '

In [27]:
generate_text(model, tokenizer, init_sentence="<start> if you")

'<start> if you re not ready for <unk> . woah oh oh ! the road of life is rocky <end> '

In [29]:
generate_text(model, tokenizer, init_sentence="<start> you are")

'<start> you are the only thing that keeps me goin <end> '

# 최종적으론 validation loss가 0.7722645401954651로 약 0.77로 나왔다.

----------------------------------------------------------------------------------------------------------------------

# 회고

이번 프로젝트에서는 Lyricst AI 를 만들어 보았다.

이번에도 역시나 텍스트 데이터 전처리 과정의 중요성에 대해 다시 한번 배웠다. 또한, 학습하는 벡터 수가 많아서인지 파라미터 갯수 또한 많아서인지 학습량이 많이 필요한 것 같다. 그리고 모델을 만들 때는 최대한 사람의 인식 패턴과 비슷하게 만들어야 더 좋은 성능을 내는 것 같습니다.

최종 결과로는 validation loss가 약 0.77로 나왔으며, generate_text 함수로 문장 형성을 체크했는데 작곡을 잘 하는 좋은 성능을 보여줬다.
뛰어난 성능에 놀랐고, GPT-3에 감탄했다. 개인적으로는 Lyricst을 넘어서 글이 되고 논문을 쓰는 모델을 만들어보고 연구해보고 싶다.